In [ ]:
%pylab inline

import networkx as nx
import scipy.sparse
import pandas as pd
from scipy.io import loadmat
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.utils.extmath import squared_norm
from sklearn.covariance import empirical_covariance
from sklearn.model_selection import GridSearchCV, ShuffleSplit

from regain import datasets_v2 as datasets; reload(datasets)
from regain.datasets_v2 import is_pos_def, is_pos_semidef
from regain.plot import plot_graph_with_latent_variables
from regain.admm import latent_time_graph_lasso_v3_; reload(latent_time_graph_lasso_v3_)
from regain.admm import time_graph_lasso_; reload(time_graph_lasso_);
from regain.admm import latent_graph_lasso_; reload(latent_graph_lasso_);
import regain.utils; reload(regain.utils)
from regain.utils import error_norm, structure_error, error_rank
import time

# Generation of tables for results

In [ ]:
ours = pd.DataFrame(columns=["n_dim_obs", "time", "iterations", "F1score", "MSE_precision", "MSE_observed",
                             "MSE_latent","mean_rank_error"])
hallac = pd.DataFrame(columns=ours.columns)
chandresekeran = pd.DataFrame(columns=ours.columns)
friedman = pd.DataFrame(columns=ours.columns)

In [3]:
%load_ext pymatbridge
#%matlab cd /home/fede/Downloads/lvglasso-pub/
%matlab cd /home/veronica/src/lvglasso-pub/

Starting MATLAB on ZMQ socket ipc:///tmp/pymatbridge-795851ca-5de0-4ba5-9717-4fef20170b6c
Send 'exit' command to kill the server
........MATLAB started and connected!


# Performances of the different algorithms

In [4]:
def ltgl_results(data_grid, dataframe, K, K_obs, ells, tau=np.linspace(0.01, 0.8, 10), eta=np.linspace(0.01, 0.8, 10),
                   beta=np.linspace(0.01, 0.8, 10), alpha=np.linspace(0.01, 0.2, 10)):
    
    mdl = latent_time_graph_lasso_v3_.LatentTimeGraphLasso(
        bypass_transpose=True, assume_centered=0, verbose=0, rtol=1e-5, tol=1e-5,
        max_iter=500, rho=1./ np.sqrt(np.array(data_list).shape[1]))
    params = {'tau': tau,
              'eta':eta,
              'alpha':alpha, 
              'beta':beta}
    ltgl = GridSearchCV(mdl, params, cv=ShuffleSplit(3, test_size=.5)).fit(data_grid)
    alpha = ltgl.best_params_['alpha']
    tau = ltgl.best_params_['tau']
    beta = ltgl.best_params_['beta']
    eta = ltgl.best_params_['eta']
    
    start_time = time.time()
    ll = latent_time_graph_lasso_v3_.LatentTimeGraphLasso(
        bypass_transpose=False, assume_centered=True, verbose=0, rtol=1e-5, tol=1e-5, max_iter=500,
        rho=1. / np.array(data_list).shape[1], eta=eta, beta=beta, tau=tau, alpha=alpha).fit(data_grid)
    time = time.time() - start_time()
    iterations = ll.n_iter_
    F1score = structure_error(K, ll.precision_)['F1score']
    MSE_observed = error_norm(K_obs, ll.precision_ - ll.latent_)
    MSE_precision = error_norm(K, ll.precision_)
    MSE_latent = error_norm(ells, ll.latent_)
    mean_rank_error = error_rank(ells, ll.latent_)

    res = pd.DataFrame([K.shape[0], time, iterations, F1score,
                        MSE_precision, MSE_observed, MSE_latent, 
                        mean_rank_error], columns=ours.columns.values)
    dataframe = dataframe.append(res, ignore_index=True)
    return dataframe, alpha, tau, beta, eta

In [5]:
def hallac_results(data_grid, dataframe, K, K_obs, ells, tau, eta, beta, alpha):
        return None

In [6]:
from sklearn.covariance import GraphLasso
def glasso_results(data_grid, dataframe, K, K_obs, ells, alpha):
    gl = GraphLasso(alpha=alpha, mode='cd',assume_centered=True)
    
    start_time = time.time()
    iters = []
    precisions = []
    for d in data_grid.transpose(2,0,1):
        gl.fit(d)
        iters.append(gl.n_iter_)
        precisions.append(gl.precision_)
    time = time.time() - start_time()
    iterations = np.max(iters)
    
    F1score = structure_error(K, precisions)['F1score']
    MSE_observed = None
    MSE_precision = error_norm(K, precisions)
    MSE_latent = None
    mean_rank_error = None

    res = pd.DataFrame([K.shape[0], time, iterations, F1score,
                        MSE_precision, MSE_observed, MSE_latent, 
                        mean_rank_error], columns=ours.columns.values)
    dataframe = dataframe.append(res, ignore_index=True)
    return dataframe

In [ ]:
from pymatbridge import Matlab
def chandresekeran_results(data_grid, dataframe, K, K_obs, ells, tau, alpha):
   
    emp_list = np.array([empirical_covariance(x, assume_centered=True)
                        for x in data_grid.transpose(2,0,1)])
    start_time = time.time()
    
    mlab = Matlab()
    mlab.start()
    results = mlab.run_code('a=1;')
    mlab.stop()
    

# Run experiments

In [ ]:
data = []
taus = []
alphas = []
for n_dim_obs in np.logspace(1,5,1):
    n_samples = 100
    n_dim_lat = 7
    T=10

    res = datasets.generate_dataset(
         mode='fixed', n_samples=n_samples, n_dim_lat=n_dim_lat, n_dim_obs=n_dim_obs,  T=T, epsilon=1e-2)
    data_list = res["data_list"]
    K = res["thetas"]
    K_obs = res["thetas_observed"]
    ells = res["ells"]
    data_grid = np.array(data_list).transpose(1,2,0)  # to use it later for grid search
    data.append(data_grid)
    ours, alpha, tau, beta, eta = ltgl_results(data_grid, ours, K, K_obs, ells)
    taus.append(tau)
    alhpas.append(alpha)
    friedman = glasso_results(data_grid, friedman,  K, K_obs, ells, tau, eta, beta, alpha)
    #hallac = hallac_results(data_grid, hallac, K, K_obs, ells, tau, eta, beta, alpha)
    

# Results chandresekeran

In [ ]:
emps = []
for d in data:
    emp_list = np.array([empirical_covariance(x, assume_centered=True)
                        for x in d.transpose(2,0,1)])
    emps.append(emp_list)

In [ ]:
%%matlab -i emps,alpha,tau -o R,S,L,obj,res,iter
%alpha = 0.05;
%beta = 0.25;
opts.continuation = 1; opts.num_continuation = 0;
opts.eta = sqrt(100); opts.muf = 1e-6;
opts.maxiter = 500; opts.stoptol = 1e-5; 
opts.over_relax_par = 1;

R = cell(1, size(emp_list,3));
S = cell(1, size(emp_list,3));
L = cell(1, size(emp_list,3));
obj = cell(1, size(emp_list,3));
res = cell(1, size(emp_list,3));
iter = cell(1, size(emp_list,3));
for i=1:size(emp_list,3)
    cov = emp_list{i};
    n = size(cov,1);opts.mu = n;
    tic; out_B = ADMM_B(cov,alpha,tau,opts); solve_B = toc;
    %%fprintf('ADMM_B: obj: %e, iter: %d, cpu: %3.1f \n',out_B.obj,out_B.iter,solve_B);
    R{i} = out_B.R;
    S{i} = out_B.S;
    L{i} = out_B.L;
    obj{i} = out_B.obj;
    res{i} = out_B.resid;
    iter{i} = out_B.iter;
end

In [ ]:
   
R = np.array(R)
L = np.array(L)
iters = np.arrau(iter)
R_ar = np.array([R[i*R.shape[1]:(i+1)*R.shape[1]] for i in range(R.shape[0]/R.shape[1])])
L_ar = np.array([L[i*L.shape[1]:(i+1)*L.shape[1]] for i in range(L.shape[0]/L.shape[1])])
iters_ar = np.array([iters[i*iters.shape[1]:(i+1)*iters.shape[1]]
                     for i in range(iters.shape[0]/iters.shape[1])])
print(iters_ar)
iterations = np.max(iters_ar)
F1score = structure_error(K, R_ar+L_ar)['F1score']
MSE_observed = error_norm(K_obs, R_ar)
MSE_precision = error_norm(K, R_ar+L_ar)
MSE_latent = error_norm(ells, L_ar)
mean_rank_error = error_rank(ells, L_ar)

res = pd.DataFrame([K.shape[0], time, iterations, F1score,
                    MSE_precision, MSE_observed, MSE_latent, 
                    mean_rank_error], columns=ours.columns.values)
dataframe = dataframe.append(res, ignore_index=True)
return dataframe

In [ ]:
print(ours)
print(friedman)